In [1]:
pwd

u'/Users/SD'

In [2]:
cd Downloads/

/Users/SD/Downloads


In [4]:
cd yelp_dataset_challenge_academic_dataset/

/Users/SD/Downloads/yelp_dataset_challenge_academic_dataset


In [9]:
import json
import pickle
reviews = []
i = 0
with open('yelp_academic_dataset_review.json') as f:
    for line in f:
        reviews.append(json.loads(line))
        if i == 250000:
            with open('subset-reviews-0.pkl','w') as fw:
                pickle.dump(reviews,fw)
                break
        i += 1

In [10]:
ls -l

total 5160856
-rw-r-----@ 1 SD  staff      397375 Jan 13 11:45 Dataset_Challenge_Academic_Dataset_Agreement.pdf
-rw-r-----@ 1 SD  staff      238869 Jan 13 11:45 Yelp_Dataset_Challenge_Terms_round_7.pdf
-rw-r--r--  1 SD  staff   251301066 Apr 24 14:11 subset-reviews-0.pkl
-rw-r--r--@ 1 SD  staff    69035997 Jan  7 20:34 yelp_academic_dataset_business.json
-rw-r--r--@ 1 SD  staff    25516317 Jan  7 20:38 yelp_academic_dataset_checkin.json
-rw-r--r--@ 1 SD  staff  1940765746 Jan  7 20:37 yelp_academic_dataset_review.json
-rw-r--r--@ 1 SD  staff   118706610 Jan  7 20:38 yelp_academic_dataset_tip.json
-rw-r--r--@ 1 SD  staff   236377074 Jan  7 20:38 yelp_academic_dataset_user.json


In [11]:
from pprint import pprint
pprint(reviews[0])

{u'business_id': u'5UmKMjUEUNdYWqANhGckJw',
 u'date': u'2012-08-01',
 u'review_id': u'Ya85v4eqdd6k9Od8HbQjyA',
 u'stars': 4,
 u'text': u'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.',
 u'type': u'review',
 u'user_id': u'PUFPaY9KxDAcGqfsorJp3Q',
 u'votes': {u'cool': 0, u'funny': 0, u'useful': 0}}


In [13]:
import gensim

In [40]:
import re
from gensim.models.doc2vec import LabeledSentence

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


class MySentences(object):
    def __init__(self, listname):
        self.listname = listname

    def __iter__(self):
        for review in self.listname:
            clean_review = clean_str(review['text'])
            yield LabeledSentence(words=clean_review.split(), tags=str(review['stars']))
            
sentences = MySentences(reviews[:250]) # a memory-friendly iterator
model = gensim.models.Doc2Vec(sentences)

In [52]:
model.most_similar(['1'])

[(u'no', 0.8778679966926575),
 (u'maybe', 0.8742163777351379),
 (u'garlic', 0.8704522252082825),
 (u'peppers', 0.867580235004425),
 (u'2', 0.8627973198890686),
 (u'decent', 0.8350401520729065),
 (u'two', 0.8230847120285034),
 (u'eggs', 0.8163115978240967),
 (u'don', 0.8115946650505066),
 (u'italian', 0.8049217462539673)]

In [28]:
len(model.vocab)

39081

In [29]:
import nltk
import nl